# Pipeline

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

dataset = pd.read_csv('../Data/SMSSpamCollection.tsv', sep='\t', names=['label', 'message'])

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

dataset['message_length'] = dataset['message'].apply(lambda x: len(x) - x.count(" "))
dataset['punct%'] = dataset['message'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(dataset['message'])

X_features = pd.concat([dataset['message_length'], dataset['punct%'], pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vect.get_feature_names_out())], axis=1)
X_features.head()

,message_length,punct%,,0,008704050406,0089mi,0121,01223585236,01223585334,0125698789,...,zindgi,zoe,zogtoriu,zoom,zouk,zyada,é,ü,üll,〨ud
0,160,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Build Grid Search with Cross Validation

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150],
    'max_depth': [7, 17, 15],
    'learning_rate': [0.1]
}

gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_features, dataset['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,262.279967,59.159766,0.266497,0.051568,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.965889,0.981149,0.971275,0.966757,0.966757,0.970365,0.005713,1
3,436.472173,12.156020,0.270514,0.049719,0.1,17,150,"{'learning_rate': 0.1, 'max_depth': 17, 'n_est...",0.965889,0.976661,0.971275,0.967655,0.969452,0.970186,0.003702,2
2,261.529148,3.920421,0.221321,0.011404,0.1,17,100,"{'learning_rate': 0.1, 'max_depth': 17, 'n_est...",0.964093,0.975763,0.968582,0.970350,0.971249,0.970007,0.003789,3
4,263.893043,18.308446,0.235845,0.035173,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964991,0.974865,0.970377,0.968553,0.969452,0.969648,0.003184,4
5,451.414805,12.380184,0.200283,0.030505,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.965889,0.976661,0.969479,0.968553,0.966757,0.969468,0.003814,5
